In [ ]:
# 시나리오 설명
'''
김싸피가 속한 마케팅 부서는 고객의 누적 가치를 기반으로
특정 기준 이상을 만족하는 VIP 고객에게 특별 쿠폰을 발송하려 합니다.
고객 구매 내역에서 Customer Lifetime Value를 합산하고,
기준 금액 이상인 경우 "alert": "yes"로 표시해 CSV로 저장하세요.
사용 데이터셋 : Global Customer Loyalty Program
https://www.kaggle.com/datasets/imadali595/global-customer-loyalty-program?utm_source=chatgpt.com
'''

In [ ]:
# 요구 사항
'''
기준 금액은 50,000으로 고정
groupby → sum → reset_index → merge 순서로 구성
alert 컬럼은 "yes" 또는 "no"로 구분
결과 저장 전 os.makedirs()를 활용해 디렉토리 생성 필요
'''

In [2]:
# 1. 라이브러리 불러오기
import pandas as pd
import os

In [3]:
# 2. CSV 데이터 불러오기
# 고객 데이터를 로딩하고 상위 5개 행을 출력해 구조를 확인합니다
df = pd.read_csv("../data/CustomerLoyaltyProgram.csv")
print(df.head(5))

   Loyalty# First Name Last Name  Customer Name  Country Province or State  \
0  837810.0    Abigail     Goris  Abigail Goris   Canada  British Columbia   
1  707928.0        Ada    Glaude     Ada Glaude  Germany            Bremen   
2  901442.0        Ada   Staback    Ada Staback   Canada          Manitoba   
3  237303.0       Adam     Boner     Adam Boner   Canada           Ontario   
4  348835.0       Adam  Dunnegan  Adam Dunnegan   Canada          Manitoba   

           City   Latitude   Longitude Postal code  ... LoyaltyStatus  \
0  Dawson Creek  55.759627 -120.237662     U5I 4F1  ...        Bronze   
1        Bremen  53.079296    8.801694       28759  ...        Bronze   
2      Winnipeg  49.895136  -97.138374     R2C 0M5  ...        Bronze   
3       Sudbury  46.522099  -80.953035     M5V 1G5  ...        Bronze   
4      Winnipeg  49.895136  -97.138374     R2C 0M5  ...        Bronze   

                Product Line Coupon Response  Count Quantity Sold  \
0  Computers and Home O

In [4]:
# 3. 고객별 누적 가치 집계
# 'Loyalty#' 기준으로 'Customer Lifetime Value'를 합산합니다
summary = df.groupby("Loyalty#")["Customer Lifetime Value"].sum().reset_index()
summary.rename(columns={"Customer Lifetime Value": "누적가치"}, inplace=True)

In [5]:
# 4. 고객 이름 연결
# 고유한 'Loyalty#'에 해당하는 고객 이름을 병합합니다
name_map = df.drop_duplicates("Loyalty#")[["Loyalty#", "Customer Name"]]
summary = summary.merge(name_map, on="Loyalty#", how="inner")

In [6]:
summary.head()

,Loyalty#,누적가치,Customer Name
0,100018.0,7919.20,Mina Smida
1,100055.0,39769.56,Sadie Straseskie
2,100102.0,2887.74,Rigoberto Palacio
3,100119.0,5146.90,Rheba Stonum
4,100124.0,2248.45,Soo Schier


In [7]:
# 5. 알림 조건 적용
# 누적 가치가 50000 이상인 경우 'alert' 컬럼에 "yes" 표시합니다
threshold = 50000
summary["alert"] = summary["누적가치"].apply(lambda x: "yes" if x >= threshold else "no")

In [8]:
# 6. 결과 저장
# 결과를 저장할 디렉토리를 생성한 후, CSV 파일로 저장합니다
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)
summary.to_csv("../output/alert_lifetime_summary.csv", index=False)

In [9]:
# 7. 결과 확인
# 누적 가치가 높은 순으로 정렬된 데이터를 확인합니다
summary.sort_values(by="누적가치", ascending=False).head( )

,Loyalty#,누적가치,Customer Name,alert
51619,834417.0,249976.14,Rocio Millisor,yes
29265,514396.0,193856.28,Teodoro Fiume,yes
57091,912633.0,189820.81,Aleisha Andrino,yes
47895,781219.0,188964.84,Elenore Restifo,yes
61334,972502.0,177232.92,Johnette Zutell,yes
